In [27]:
from __future__ import print_function

In [28]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [43]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import matplotlib.pyplot as plt
import PIL

In [44]:
use_cuda = torch.cuda.is_available()

In [45]:
from dataset import dataset
from AlexNet import AlexNet
from train_test import start_train_test
#from autoaugment import dataset

In [47]:
#trainloader, testloader, outputs, inputs = dataset('dog-breed')
#print ('Output classes: {}\nInput channels: {}'.format(outputs, inputs))


rootdir='/Users/luciaeve/Documents/EMSE/COURSES/VDL/Very-Deep-Learning-CNN/test_data/'
transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((224,224)),
    torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR)
])

dataset = torchvision.datasets.ImageFolder(rootdir, transform=transforms)
display(dataset)






Dataset ImageFolder
    Number of datapoints: 10357
    Root Location: /Users/luciaeve/Documents/EMSE/COURSES/VDL/Very-Deep-Learning-CNN/test_data/
    Transforms (if any): Compose(
                             Resize(size=(224, 224), interpolation=PIL.Image.BILINEAR)
                             ColorJitter(brightness=0, contrast=0, saturation=0.05, hue=0.05)
                             RandomHorizontalFlip(p=0.5)
                             RandomRotation(degrees=(-20, 20), resample=2, expand=False)
                         )
    Target Transforms (if any): None

In [8]:
net = AlexNet(num_classes = outputs,inputs=inputs)
file_name = 'alexnet-'

In [9]:
if use_cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

In [10]:
criterion = nn.CrossEntropyLoss()

In [ ]:
train_loss, test_loss = start_train_test(net, trainloader, testloader, criterion)

In [ ]:
plt.plot(train_loss)
plt.ylabel('Train Loss')
plt.show()

In [ ]:
plt.plot(test_loss)
plt.ylabel('Test Loss')
plt.show()

In [ ]:
# exercise 4 
from autoaugment import get_transform_policies

augmented_training_data = torchvision.datasets.ImageFolder('data/train', 
                                                      transform=get_transform_policies())
# inject the augmented training data 
